In [3]:
!pip install tensorflow

In [5]:
!pip install kaggle

In [7]:
import os

# Set the Kaggle API key (replace with your actual key from the kaggle.json file)
os.environ['KAGGLE_USERNAME'] = 'nathancarbucks17'
os.environ['KAGGLE_KEY'] = 'bea51467a449eb6e10f869ad92d7547c'


In [9]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other




  0%|          | 0.00/2.29G [00:00<?, ?B/s]
  0%|          | 1.00M/2.29G [00:00<09:59, 4.10MB/s]
  0%|          | 3.00M/2.29G [00:00<05:02, 8.12MB/s]
  0%|          | 6.00M/2.29G [00:00<02:49, 14.5MB/s]
  0%|          | 9.00M/2.29G [00:00<03:29, 11.7MB/s]
  1%|          | 15.0M/2.29G [00:00<01:52, 21.8MB/s]
  1%|          | 18.0M/2.29G [00:01<02:07, 19.2MB/s]
  1%|1         | 29.0M/2.29G [00:01<01:08, 35.4MB/s]
  2%|1         | 36.0M/2.29G [00:01<00:56, 42.9MB/s]
  2%|1         | 41.0M/2.29G [00:01<00:58, 41.0MB/s]
  2%|1         | 46.0M/2.29G [00:01<01:02, 38.6MB/s]
  2%|2         | 51.0M/2.29G [00:01<01:14, 32.5MB/s]
  2%|2         | 55.0M/2.29G [00:02<01:14, 32.3MB/s]
  3%|2         | 59.0M/2.29G [00:02<01:23, 28.7MB/s]
  3%|2         | 62.0M/2.29G [00:02<01:23, 28.6MB/s]
  3%|2         | 65.0M/2.29G [00:02<01:46, 22.5MB/s]
  3%|2         | 69.0M/2.29G [00:02<01:32, 26.0MB/s]
  3%|3         | 73.0M/2.29G [00:02<01:23, 28.6MB/s]
  3%|3         | 77.0M/2.29G [00:03<01:26, 27.7MB/s]
 

In [13]:
import zipfile

# Path to the downloaded zip file
zip_file = 'chest-xray-pneumonia.zip'  # Adjust this if your file name is different
extract_dir = 'chest_xray'  # Directory where you want to extract the contents

# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Dataset extracted to {extract_dir}")


Dataset extracted to chest_xray


In [15]:
import os

# List the contents of the extracted directory
extracted_files = os.listdir(extract_dir)
print(f"Files in extracted directory: {extracted_files}")


Files in extracted directory: ['chest_xray']


In [27]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the directories for training and validation data
#train_dir ='chest_xray\train'
#test_dir = 'chest_xray\test'

train_dir ='chest_xray/chest_xray/train'
test_dir = 'chest_xray/chest_xray/test'

# Image preprocessing and augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize the images to [0, 1]
    rotation_range=40,    # Random rotation
    width_shift_range=0.2, # Random horizontal shift
    height_shift_range=0.2, # Random vertical shift
    shear_range=0.2,      # Random shear
    zoom_range=0.2,       # Random zoom
    horizontal_flip=True, # Random horizontal flip
    fill_mode='nearest'   # Fill any missing pixels after transformations
)

# For validation data, just rescale the images (no augmentation)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Load the data from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='binary'      # Use 'binary' for binary classification (normal vs. pneumonia)
)

valid_generator = valid_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'      # Use 'binary' for binary classification
)


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [29]:
# Build the CNN model
model = tf.keras.Sequential([
    # Convolutional layer with ReLU activation
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Flatten the 3D tensor to 1D
    tf.keras.layers.Flatten(),
    
    # Fully connected layer
    tf.keras.layers.Dense(512, activation='relu'),
    
    # Output layer with a sigmoid activation (binary classification)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Model summary to see the architecture
model.summary()


C:\Users\Nathan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary crossentropy for binary classification
              metrics=['accuracy'])


In [35]:
# Train the model
history = model.fit(
    train_generator,  # Training data
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Total batches per epoch
    epochs=10,        # Number of epochs
    validation_data=valid_generator,  # Validation data
    validation_steps=valid_generator.samples // valid_generator.batch_size  # Batches per validation epoch


C:\Users\Nathan\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 918ms/step - accuracy: 0.7742 - loss: 0.5382

C:\Users\Nathan\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 163s 989ms/step - accuracy: 0.7744 - loss: 0.5377 - val_accuracy: 0.7747 - val_loss: 0.4648
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 123s 753ms/step - accuracy: 0.8259 - loss: 0.3502 - val_accuracy: 0.8618 - val_loss: 0.3340
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 124s 762ms/step - accuracy: 0.8626 - loss: 0.3046 - val_accuracy: 0.8503 - val_loss: 0.3502
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 121s 741ms/step - accuracy: 0.8684 - loss: 0.2942 - val_accuracy: 0.8586 - val_loss: 0.3740
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 123s 752ms/step - accuracy: 0.8945 - loss: 0.2413 - val_accuracy: 0.8849 - val_loss: 0.2977
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 125s 768ms/step - accuracy: 0.9046 - loss: 0.2279 - val_accuracy: 0.8536 - val_loss: 0.3898
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 129s 788ms/step - accuracy: 0.9133 - loss: 0.2087 - val_accuracy: 0.8569 - val_loss: 0.3667
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 124s 760ms/step - accuracy: 0.9186 - loss: 0.20

In [73]:
!pip install torch

   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
    --------------------------------------- 3.9/203.0 MB 21.3 MB/s eta 0:00:10
   - -------------------------------------- 8.9/203.0 MB 21.3 MB/s eta 0:00:10
   -- ------------------------------------- 13.4/203.0 MB 21.5 MB/s eta 0:00:09
   --- ------------------------------------ 17.0/203.0 MB 21.9 MB/s eta 0:00:09
   ---- ----------------------------------- 23.1/203.0 MB 22.1 MB/s eta 0:00:09
   ----- ---------------------------------- 28.0/203.0 MB 22.0 MB/s eta 0:00:08
   ------ --------------------------------- 32.5/203.0 MB 22.0 MB/s eta 0:00:08
   ------- -------------------------------- 37.0/203.0 MB 21.9 MB/s eta 0:00:08
   -------- ------------------------------- 41.7/203.0 MB 21.9 MB/s eta 0:00:08
   --------- ------------------------------ 46.7/203.0 MB 22.0 MB/s eta 0:00:08
   ---------- ----------------------------- 51.1/203.0 MB 22.0 MB/s eta 0:00:07
   ---------- ----------------------------- 55.8/20

In [79]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

# Check if the directory exists, if not create it
save_dir = r'C:\Users\Nathan\Documents\models'  # Use raw string to avoid issues with backslashes
if not os.path.exists(save_dir):
    os.makedirs(save_dir)  # Create the directory if it doesn't exist

# Define a simple model (replace with your actual model)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 26 * 26, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc1(x)
        return x

# Instantiate the model (use your own trained model here)
model = SimpleModel()

# Example: Create dummy data and a simple training loop (if needed)
# This is just for demonstration and not necessary if your model is already trained.
dummy_input = torch.randn(1, 1, 28, 28)  # Batch size of 1, 1 channel, 28x28 image size
dummy_target = torch.randint(0, 10, (1,))  # Random target for classification

# Define a simple loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Simulate a training step (this is just for demonstration purposes)
optimizer.zero_grad()
output = model(dummy_input)
loss = criterion(output, dummy_target)
loss.backward()
optimizer.step()

# Save the model's state_dict
model_save_path = os.path.join(save_dir, 'model_for_scifair.pt')
torch.save(model.state_dict(), model_save_path)

print(f"Model saved as 'model_for_scifair.pt' in directory {save_dir}")


Model saved as 'model_for_scifair.pt' in directory C:\Users\Nathan\Documents\models
